In [ ]:
#default_exp schema

# Schema module
Schema of the database object

In [ ]:
#hide
import pickle, os

os.environ['DATABASE_TABLE_NAME'] = 'product-table-dev-manual'
os.environ['REGION'] = 'ap-southeast-1'
os.environ['INVENTORY_BUCKET_NAME'] = 'product-bucket-dev-manual'
os.environ['INPUT_BUCKET_NAME'] = 'input-product-bucket-dev-manual'
os.environ['DAX_ENDPOINT'] = 'longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111'
os.environ['LINEKEY'] = '2uAfV4AoYglUGmKTAk2xNOm0aV2Ufgh1BQPvQl9vJd4'
REGION = 'ap-southeast-1'

In [ ]:
#export
import os

DATABASE_TABLE_NAME = os.environ.get('DATABASE_TABLE_NAME')
INVENTORY_BUCKET_NAME = os.environ.get('INVENTORY_BUCKET_NAME')
INPUT_BUCKET_NAME = os.environ.get('INPUT_BUCKET_NAME')
REGION = os.environ.get('REGION') or 'ap-southeast-1'
ACCESS_KEY_ID = os.environ.get('USER') or None
SECRET_ACCESS_KEY = os.environ.get('PW') or None
LINEKEY= os.environ.get('LINEKEY')
  
try:
  DAX_ENDPOINT = os.environ['DAX_ENDPOINT']
except KeyError as e:
  DAX_ENDPOINT = None
  print(f'dax endpoint missing {e}')
  
print(DAX_ENDPOINT)

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


In [ ]:
from villaProductDatabase.database import ProductDatabase
from villaProductDatabase.query import Querier

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


In [ ]:
#export
from nicHelper.wrappers import add_method, add_class_method, add_static_method
from pynamodb.attributes import UnicodeAttribute, NumberAttribute, JSONAttribute, BooleanAttribute, BinaryAttribute
from pynamodb.indexes import GlobalSecondaryIndex, AllProjection
from pynamodb.models import Model
from pprint import pformat

## Create index helper function

In [ ]:
#export
def createIndex(name, rangeKeyName= None, indexName = None, HashKeyType = UnicodeAttribute, RangeKeyType = UnicodeAttribute):
  class ReturnSecondaryIndex(GlobalSecondaryIndex):
    class Meta:
      index_name = indexName or name
      projection = AllProjection()
      dax_read_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
      dax_write_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
      read_capacity_units = 1
      write_capacity_units = 1
  setattr(ReturnSecondaryIndex, name, HashKeyType(hash_key = True))
  if rangeKeyName:
    setattr(ReturnSecondaryIndex, rangeKeyName, RangeKeyType(range_key = True))
  return ReturnSecondaryIndex()

In [ ]:
#export
class KeySchema(Model):
  iprcode = NumberAttribute(hash_key=True, default = 0)
  cprcode = NumberAttribute(default = 0, range_key = True)
  oprcode = NumberAttribute(default = 0)
  pr_dpcode = UnicodeAttribute(default = 'none')
  pr_barcode = UnicodeAttribute(default = 'none')
  pr_barcode2 = UnicodeAttribute(default = 'none')
  pr_sucode1 = UnicodeAttribute(default = 'none')
  pr_suref3 = UnicodeAttribute(default= 'none')
  pr_sa_method = UnicodeAttribute(default= 'none')
  sellingPrice = NumberAttribute(default = 0)
  lastUpdate = NumberAttribute( default = 0)
  needsUpdate = UnicodeAttribute(default = 'Y')
  data = JSONAttribute()

  # indexes
  needsUpdateIndex = createIndex('needsUpdate','sellingPrice')
  cprcodeIndex = createIndex('cprcode', 'sellingPrice')
  oprcodeIndex = createIndex('oprcode', 'sellingPrice')
  pr_dpcodeIndex = createIndex('pr_dpcode', 'sellingPrice')
  pr_barcodeIndex = createIndex('pr_barcode', 'sellingPrice')
  pr_barcode2Index = createIndex('pr_barcode2', 'sellingPrice')
  pr_suref3Index = createIndex('pr_suref3', 'sellingPrice')
  pr_sa_methodIndex = createIndex('pr_sa_method', 'sellingPrice')
  cprcodeOnlyIndex = createIndex('cprcode', indexName='cprcodeOnly')
  
  def __repr__(self):
    return pformat(vars(self)["attribute_values"])

In [ ]:
class Tester(Querier, KeySchema):
  class Meta:
    aws_access_key_id = ACCESS_KEY_ID
    aws_secret_access_key = SECRET_ACCESS_KEY
    table_name = DATABASE_TABLE_NAME
    region = REGION
    billing_mode='PAY_PER_REQUEST'
    dax_read_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
    dax_write_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
  pass

In [ ]:
list(Tester.scan(limit=1))

[{'cprcode': 225407,
  'data': {'SaleMode_unit': 'ITEM',
           'ba_nprice': '25.00',
           'content_en': '0225407 0225407 KONJAC LINGUINI  '
                         '0225407-8858661102531',
           'content_th': 'บุกเส้นแบน ตราโมกุ   MOKU MOKU',
           'cprcode': 225407,
           'hema_brand_en': '',
           'hema_brand_th': '',
           'hema_name_en': 'KONJAC LINGUINI',
           'hema_name_th': 'บุกเส้นแบน ตราโมกุ',
           'hema_sizedesc': '',
           'iprcode': 225407,
           'master_online': 'Y',
           'online_category_l1_en': 'Delicatessen & Eggs',
           'online_category_l1_th': 'อาหารสำเร็จรูปและไข่',
           'online_category_l2_en': 'Ready Meals',
           'online_category_l2_th': 'อาหารพร้อมทาน',
           'online_category_l3_en': '',
           'online_category_l3_th': '',
           'oprcode': 225407,
           'ordertype': 'Y',
           'pr_abb': 'MOKU',
           'pr_active': 'Y',
           'pr_barcode': 0,
        